In [16]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd

In [17]:
def scrape_rockfm_page(base_url='https://onlineradiobox.com/es/rockfm/playlist/'):
    # Set headers to mimic a real browser
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

    songs = []
    today = datetime.today().date()
    
    for page_idx in range(1, 8):
        
        idx = '' if page_idx == 0 else str(page_idx)
        
        date = today - timedelta(days=page_idx)
        date = date.strftime("%Y-%m-%d")
     
        page_url = base_url + idx + '?cs=es.rockfm'
    
        # Fetch the content of the webpage
        response = requests.get(page_url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find the table
            table = soup.find("table", class_="tablelist-schedule")

            # Extract rows from the table
            rows = table.find_all("tr")
 
            for row in rows:
                # Extract time
                time_cell = row.find("td", class_="tablelist-schedule__time")
                time_text = time_cell.find("span", class_="time--schedule").text.strip() if time_cell else "N/A"

                # Extract track info
                track_cell = row.find("td", class_="track_history_item")
                try:
                    track_text = track_cell.find("a").text.strip() if track_cell else ''
                except:
                    track_text = track_cell.text.strip() if track_cell else ''
                    
                artist = track_text.split(' - ')[0]
                song = track_text.split(' - ')[1]
                
                # List with one song's data
                song = [date, time_text, artist, song]
                songs.append(song)
                songs = songs[::-1]
        
        # Dataframe with all songs data
        songs_df = pd.DataFrame(songs, columns=["Date", "Time", "Artist", "Song"])
                      
    return songs_df


# URL to scrape
base_url = 'https://onlineradiobox.com/es/rockfm/playlist/'

# Scrape the songs data
songs = scrape_rockfm_page(base_url)
songs



,Date,Time,Artist,Song
0,2024-12-16,00:00,AEROSMITH,COME TOGETHER
1,2024-12-16,00:07,DOVER,CHERRY LEE
2,2024-12-16,00:14,FACES,STAY WITH ME
3,2024-12-16,00:22,DEF LEPPARD,LOVE BITES
4,2024-12-16,00:32,LOQUILLO Y TROGLODITAS,CADILLAC SOLITARIO (Remaster 30 Aniversario)
...,...,...,...,...
2036,2024-12-16,00:35,THE BEATLES,HEY JUDE
2037,2024-12-16,00:28,NO DOUBT,JUST A GIRL
2038,2024-12-16,00:19,FOO FIGHTERS,BEST OF YOU
2039,2024-12-16,00:10,FRANZ FERDINAND,DO YOU WANT TO


In [19]:
# Get the number of unique artists
unique_artists_count = songs["Artist"].nunique()
unique_artists_count

261

In [24]:
# Get unique combinations of Artist and Song
unique_songs = songs[["Artist", "Song"]].nunique()
unique_songs

Artist    261
Song      598
dtype: int64

In [30]:
# Group by Artist and Song, then count the occurrences of each combination
combination_counts = (songs.groupby(["Artist", "Song"])
                           .size()
                           .reset_index(name="Count")
                           .sort_values(by="Count", ascending=False))

# Display the result
combination_counts_sorted

,Artist,Song,Count
601,ZZ TOP,LA GRANGE,9
552,THE WHO,WHO ARE YOU,8
269,JOAN JETT & BLACKHEARTS,I LOVE ROCK AND ROLL,8
472,T REX,GET IT ON,8
168,ELVIS PRESLEY,BURNING LOVE,8
...,...,...,...
133,DAVID BOWIE,ZIGGY STARDUST,1
130,DAVID BOWIE,QUEEN BITCH,1
129,DAVID BOWIE,LIFE ON MARS,1
458,STEVE MILLER BAND,FLY LIKE AN EAGLE,1


In [33]:
def song_data(df, artist, song):
    song_df = df[(df['Artist'] == artist) & (df['Song'] == song)]
    song_df_sorted = song_df.sort_values(by=["Date", "Time"], ascending=[True, True])
    return song_df_sorted

In [35]:
song_la_grange = song_data(songs, 'ZZ TOP', 'LA GRANGE')
song_la_grange

,Date,Time,Artist,Song
52,2024-12-16,08:27,ZZ TOP,LA GRANGE
1892,2024-12-17,00:30,ZZ TOP,LA GRANGE
281,2024-12-17,22:34,ZZ TOP,LA GRANGE
1654,2024-12-18,16:03,ZZ TOP,LA GRANGE
551,2024-12-19,20:04,ZZ TOP,LA GRANGE
600,2024-12-20,05:21,ZZ TOP,LA GRANGE
1260,2024-12-21,11:21,ZZ TOP,LA GRANGE
1117,2024-12-22,07:45,ZZ TOP,LA GRANGE
1052,2024-12-22,17:20,ZZ TOP,LA GRANGE


In [36]:
def artist_data(df, artist):
    artist_df = df[df['Artist'] == artist]
    artist_df_sorted = artist_df.sort_values(by=["Date", "Time"], ascending=[True, True])
    return artist_df_sorted

In [37]:
artist_foreigner = artist_data(songs, 'FOREIGNER')
artist_foreigner

,Date,Time,Artist,Song
1952,2024-12-16,14:56,FOREIGNER,COLD AS ICE
193,2024-12-17,07:20,FOREIGNER,COLD AS ICE
207,2024-12-17,10:39,FOREIGNER,I WANT TO KNOW WHAT LOVE IS
302,2024-12-18,01:40,FOREIGNER,I WANT TO KNOW WHAT LOVE IS
1725,2024-12-18,03:32,FOREIGNER,URGENT
1620,2024-12-18,21:18,FOREIGNER,COLD AS ICE
483,2024-12-19,09:59,FOREIGNER,COLD AS ICE
1384,2024-12-20,15:59,FOREIGNER,COLD AS ICE
1318,2024-12-21,02:00,FOREIGNER,URGENT
834,2024-12-21,19:02,FOREIGNER,COLD AS ICE


In [ ]:
º